# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa6801531c0>
├── 'a' --> tensor([[ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611]])
└── 'x' --> <FastTreeValue 0x7fa680153850>
    └── 'c' --> tensor([[-0.8909, -0.4335, -0.1767,  0.5099],
                        [ 1.5233, -0.1117,  0.5797,  0.2873],
                        [-0.2449,  0.1913, -0.5707,  1.8499]])

In [4]:
t.a

tensor([[ 0.6155,  0.3371, -0.2797],
        [ 0.3713, -1.2238,  0.5611]])

In [5]:
%timeit t.a

65.7 ns ± 0.0567 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa6801531c0>
├── 'a' --> tensor([[ 0.1247, -0.1186,  1.8830],
│                   [-0.0506,  0.2396,  0.3650]])
└── 'x' --> <FastTreeValue 0x7fa680153850>
    └── 'c' --> tensor([[-0.8909, -0.4335, -0.1767,  0.5099],
                        [ 1.5233, -0.1117,  0.5797,  0.2873],
                        [-0.2449,  0.1913, -0.5707,  1.8499]])

In [7]:
%timeit t.a = new_value

74.8 ns ± 0.099 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6155,  0.3371, -0.2797],
               [ 0.3713, -1.2238,  0.5611]]),
    x: Batch(
           c: tensor([[-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6155,  0.3371, -0.2797],
        [ 0.3713, -1.2238,  0.5611]])

In [11]:
%timeit b.a

59 ns ± 0.128 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5207,  0.0378, -0.5081],
               [-0.0637,  0.6154, -0.0431]]),
    x: Batch(
           c: tensor([[-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.106 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

807 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 17 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa5bc5b1160>
├── 'a' --> tensor([[[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]],
│           
│                   [[ 0.6155,  0.3371, -0.2797],
│                    [ 0.3713, -1.2238,  0.5611]]])
└── 'x' --> <FastTreeValue 0x7fa6701d1f10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 84.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa5aba2bdc0>
├── 'a' --> tensor([[ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611],
│                   [ 0.6155,  0.3371, -0.2797],
│                   [ 0.3713, -1.2238,  0.5611]])
└── 'x' --> <FastTreeValue 0x7fa5aba2bf40>
    └── 'c' --> tensor([[-0.8909, -0.4335, -0.1767,  0.5099],
                        [ 1.5233, -0.1117,  0.5797,  0.2873],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 23.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.2 µs ± 96 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.8909, -0.4335, -0.1767,  0.5099],
                       [ 1.5233, -0.1117,  0.5797,  0.2873],
                       [-0.2449,  0.1913, -0.5707,  1.8499]],
              
                      [[-0.8909, -0.4335, -0.1767,  0.5099],
                       [ 1.5233, -0.1117,  0.5797,  0.2873],
                       [-0.2449,  0.1913, -0.5707,  1.8499]],
              
                      [[-0.8909, -0.4335, -0.1767,  0.5099],
                       [ 1.5233, -0.1117,  0.5797,  0.2873],
                       [-0.2449,  0.1913, -0.5707,  1.8499]],
              
                      [[-0.8909, -0.4335, -0.1767,  0.5099],
                       [ 1.5233, -0.1117,  0.5797,  0.2873],
                       [-0.2449,  0.1913, -0.5707,  1.8499]],
              
                      [[-0.8909, -0.4335, -0.1767,  0.5099],
                       [ 1.5233, -0.1117,  0.5797,  0.2873],
                       [-0.2449,  0.1913, -0.5707,  1.8499]],

In [26]:
%timeit Batch.stack(batches)

79.6 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499],
                      [-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499],
                      [-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499],
                      [-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499],
                      [-0.8909, -0.4335, -0.1767,  0.5099],
                      [ 1.5233, -0.1117,  0.5797,  0.2873],
                      [-0.2449,  0.1913, -0.5707,  1.8499],
                      [-0.8909, -0.4335, -0.1767,  0.5099],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

341 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
